In [1]:
from torch.utils.data import DataLoader
from utils.dataset import VCTKDataset



In [4]:
dataset = VCTKDataset("./dataset/VCTK",)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4,
                    collate_fn=VCTKDataset.collate_cvae)


Selected speakers: ['p272', 'p298', 'p271', 'p273', 'p232', 'p225', 'p333', 'p240', 'p297', 'p308']
speaker info: {0: 'p272: M, 23yo, Scottish (Edinburgh)', 1: 'p298: M, 19yo, Irish (Tipperary)', 2: 'p271: M, 19yo, Scottish (Fife)', 3: 'p273: M, 23yo, English (Suffolk)', 4: 'p232: M, 23yo, English (Southern England)', 5: 'p225: F, 23yo, English (Southern England)', 6: 'p333: F, 19yo, American (Indiana)', 7: 'p240: F, 21yo, English (Southern England)', 8: 'p297: F, 20yo, American (New York)', 9: 'p308: F, 18yo, American (Alabama)'}


In [6]:
for batch in loader:
    text, text_len, mel, mel_len, gate, spk = batch
    print(f"text: {text.shape}, text_len: {text_len.shape}, mel: {mel.shape}, mel_len: {mel_len.shape}, gate: {gate.shape}, spk: {spk.shape}")
    break  # Remove this line to iterate through the entire dataset
    

text: torch.Size([32, 119]), text_len: torch.Size([32]), mel: torch.Size([32, 80, 568]), mel_len: torch.Size([32]), gate: torch.Size([32, 568]), spk: torch.Size([32])
